In [13]:
from uli_init import simulate
import hoomd
import gsd
import mbuild as mb
from mbuild.formats.hoomd_simulation import create_hoomd_simulation
import hoomd.md
from hoomd.md import wall

/home/chris/miniconda3/envs/uli/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
system = simulate.System('PEEK', 1.0, 0.8, 20, 3, 'gaff')

/home/chris/miniconda3/envs/uli/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/chris/cme/forks/uli-init/uli_init/simulate.py:321: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  mass = _n * np.sum(ele.element_from_symbol(p.name).mass for p in polymer.particles())
/home/chris/miniconda3/envs/uli/lib/python3.7/site-packages/foyer/forcefield.py:449: UserWarning: No force field version number found in force field XML file.
  'No force field version number found in force field XML file.'
/home/chris/miniconda3/envs/uli/lib/python3.7/site-packages/foyer/forcefield.

In [8]:
dir(system)

/home/chris/miniconda3/envs/uli/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_calculate_L',
 '_pack',
 '_recover_mass_dist',
 '_type_system',
 '_weibull_k_expression',
 '_weibull_lambda_expression',
 'assert_dihedrals',
 'density',
 'epsilon',
 'forcefield',
 'meta',
 'molecule',
 'n_compounds',
 'para',
 'para_weight',
 'polymer_lengths',
 'remove_hydrogens',
 'seed',
 'system_mass',
 'system_mb',
 'system_pmd',
 'target_L']

In [15]:
hoomd_args = f"--single-mpi --mode={'gpu'}"
sim = hoomd.context.initialize(hoomd_args)
with sim:
    objs, refs = create_hoomd_simulation(system.system_pmd, r_cut=1.2, auto_scale=True)
        # refs = distance, mass, energy
    snap = objs[0]
    _all = hoomd.group.all()
    hoomd.md.integrate.mode_standard(dt=0.0001)
    integrator = hoomd.md.integrate.nvt(group=_all, kT=1.0, tau=0.1)
    hoomd.dump.gsd("trajectories/wall-squeeze.gsd", period=1e4, group=_all, phase=0, overwrite=True)
    integrator.randomize_velocities(seed=42);
 
    # Set up a wall:
    wall_origin = (snap.box.Lx/2, 0, 0)
    normal_vector = (-1, 0, 0)
    wall_origin2 = (-snap.box.Lx/2, 0, 0)
    normal_vector2 = (1, 0, 0)
    
    walls = wall.group(
                       wall.plane(origin=wall_origin, normal=normal_vector, inside = True),
                       wall.plane(origin=wall_origin2, normal=normal_vector2, inside = True)
                      )
    wall_force = wall.lj(walls, r_cut=snap.box.Lx/4)
    #wall_force = wall.???
    wall_force.force_coeff.set(snap.particles.types, sigma=1.0, epsilon=1.0, r_extrap=1.0)
    
    # Set up shrink:
    reduced_target_L = system.target_L / refs[0]
    reduced_init_L = system.system_pmd.box[0] / refs[0]
    target_box = [reduced_target_L]*3
    shrink_steps = 1e4
    
    x_variant = hoomd.variant.linear_interp([(0, reduced_init_L),
                                             (shrink_steps, target_box[0]*10)])
    y_variant = hoomd.variant.linear_interp([(0, reduced_init_L),
                                             (shrink_steps, target_box[1]*10)])
    z_variant = hoomd.variant.linear_interp([(0, reduced_init_L),
                                             (shrink_steps, target_box[2]*10)])
    box_updater = hoomd.update.box_resize(Lx = x_variant, Ly = y_variant, Lz = z_variant)
 
    
    


notice(2): Group "all" created containing 2080 particles
notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 2 exclusions             : 20
notice(2): Particles with 3 exclusions             : 800
notice(2): Particles with 4 exclusions             : 20
notice(2): Particles with 6 exclusions             : 100
notice(2): Particles with 7 exclusions             : 800
notice(2): Particles with 8 exclusions             : 220
notice(2): Particles with 9 exclusions             : 120
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
Processing LJ and QQ
notice(2): Group "charged" created containing 0 particles


/home/chris/miniconda3/envs/uli/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


No charged groups found, ignoring electrostatics
Processing 1-4 interactions, adjusting neighborlist exclusions
Processing harmonic bonds
Processing harmonic angles
Processing periodic torsions
HOOMD SimulationContext updated from ParmEd Structure
